# Binance vs. Lighter Price Spread Analysis (Mainnet Realtime)

This notebook fetches minute-level price data from **Binance Futures (Mainnet)** and **Lighter (Mainnet)** to analyze the spread.

**Objective:** Plot `(Binance Price - Lighter Price) / Binance Price` in real-time.
> **Note:** We use public endpoints for data fetching, so no API keys are required.

In [ ]:
import os
import sys
import asyncio
import time
import aiohttp
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from binance.client import Client as BinanceClient
from IPython.display import clear_output, display

# Add src to path
sys.path.append(os.path.abspath('src'))

%matplotlib inline

In [ ]:
# 1. Initialize Clients (Public Data Mode)
# We don't need private keys for public market data.

binance_client = BinanceClient(testnet=False) # Mainnet

# Lighter Mainnet API URL
LIGHTER_MAINNET_API = "https://mainnet.zklighter.elliot.ai"
LIGHTER_API_VERSION = "/api/v1" # Mainnet on elliot.ai uses /api/v1

# Helper to fetch public data without SDK overhead for this analysis
async def fetch_public_json(url, params=None):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "application/json"
    }
    async with aiohttp.ClientSession(headers=headers) as session:
        async with session.get(url, params=params) as resp:
            if resp.status == 200:
                return await resp.json()
            else:
                # text = await resp.text()
                # print(f"Error {resp.status} fetching {url}: {text}")
                return None

print("Clients Initialized (Mainnet).")

## 2. Real-Time Monitor
Run the cell below to start the live plotting. Stop it by clicking 'Stop' or Interrupt Kernel.

In [ ]:
async def live_spread_monitor():
    data = []
    print(f"Starting Real-Time Monitor (Stop with Interrupt)...")
    
    # Setup Plot
    # plt.ion() # Interactive mode on
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    
    try:
        while True:
            loop_start = time.time()
            now = datetime.now()
            
            # 1. Fetch Data
            b_price = None
            l_mid = None
            
            # Binance
            try:
                b_ticker = binance_client.futures_symbol_ticker(symbol="ETHUSDT")
                b_price = float(b_ticker['price'])
            except: pass
            
            # Lighter
            try:
                ob_url = f"{LIGHTER_MAINNET_API}{LIGHTER_API_VERSION}/orderBookOrders"
                # market_id=2048 for ETH/USDC
                market_id = 2048 
                params = {'market_id': market_id, 'limit': 1}
                l_ob = await fetch_public_json(ob_url, params)

                if l_ob and 'bids' in l_ob and 'asks' in l_ob:
                    if len(l_ob['bids']) > 0 and len(l_ob['asks']) > 0:
                         bid_entry = l_ob['bids'][0]
                         ask_entry = l_ob['asks'][0]
                         bid_price = float(bid_entry['price']) if isinstance(bid_entry, dict) else float(bid_entry[0])
                         ask_price = float(ask_entry['price']) if isinstance(ask_entry, dict) else float(ask_entry[0])
                         l_mid = (bid_price + ask_price) / 2
            except: pass
            
            # 2. Store & Calculate
            if b_price and l_mid:
                spread = (b_price - l_mid) / b_price
                data.append({'timestamp': now, 'binance': b_price, 'lighter': l_mid, 'spread': spread})
                
                # Keep last 1000 points
                if len(data) > 1000: data.pop(0)

            # 3. Update Plot (every ~2nd tick to avoid flicker overdrive)
            if len(data) > 2:
                df = pd.DataFrame(data).set_index('timestamp')
                
                clear_output(wait=True)
                
                # Reset Axes
                ax1.cla()
                ax2.cla()
                
                # Price Chart
                ax1.plot(df.index, df['binance'], label='Binance', color='orange')
                ax1.plot(df.index, df['lighter'], label='Lighter', color='blue')
                ax1.legend(loc='upper left')
                ax1.set_title(f"[Live] ETH Price | Binance: {b_price:.2f} | Lighter: {l_mid:.2f}")
                ax1.grid(True, alpha=0.3)
                
                # Spread Chart
                ax2.plot(df.index, df['spread'], label='Spread', color='green')
                ax2.axhline(0, color='black', linestyle='--', alpha=0.5)
                ax2.legend(loc='upper left')
                ax2.set_title(f"Rel. Spread: {spread*10000:.2f} bps")
                ax2.grid(True, alpha=0.3)
                
                display(fig)
            
            # Sleep remainder of 1s interval (loop 1Hz)
            elapsed = time.time() - loop_start
            if elapsed < 1.0:
                await asyncio.sleep(1.0 - elapsed)
                
    except KeyboardInterrupt:
        print("Stopped by User.")
        plt.close()


await live_spread_monitor()